In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from PIL import Image


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, lr_folder, hr_folder, transform=None):
        self.lr_folder = lr_folder
        self.hr_folder = hr_folder
        self.transform = transform
        self.lr_images = os.listdir(lr_folder)
        self.hr_images = os.listdir(hr_folder)

    def __len__(self):
        return len(self.lr_images)

    def __getitem__(self, idx):
        lr_image = Image.open(os.path.join(self.lr_folder, self.lr_images[idx]))
        hr_image = Image.open(os.path.join(self.hr_folder, self.hr_images[idx]))

        if self.transform:
            lr_image = self.transform(lr_image)
            hr_image = self.transform(hr_image)

        return lr_image, hr_image


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
train_lr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/train/train'
train_hr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/train/gt'
eval_lr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/val/val'
eval_hr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt'
test_lr_folder = '/kaggle/input/dlp-jan-2025-nppe-3/archive/test'

train_dataset = ImageDataset(train_lr_folder, train_hr_folder, transform=transform)
eval_dataset = ImageDataset(eval_lr_folder, eval_hr_folder, transform=transform)
test_dataset = ImageDataset(test_lr_folder, test_lr_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)
        self.relu = nn.ReLU(inplace=True)
        self.upsample = nn.Upsample(scale_factor=4, mode='bicubic', align_corners=False)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        x = self.upsample(x)
        return x

model = SRCNN().cuda()


In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for lr_images, hr_images in train_loader:
        lr_images = lr_images.cuda()
        hr_images = hr_images.cuda()

        optimizer.zero_grad()
        outputs = model(lr_images)
        loss = criterion(outputs, hr_images)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation
    model.eval()
    eval_loss = 0.0
    with torch.no_grad():
        for lr_images, hr_images in eval_loader:
            lr_images = lr_images.cuda()
            hr_images = hr_images.cuda()
            outputs = model(lr_images)
            loss = criterion(outputs, hr_images)
            eval_loss += loss.item()

    print(f'Evaluation Loss: {eval_loss/len(eval_loader)}')


In [ ]:
model.eval()
test_predictions = []
with torch.no_grad():
    for lr_images, _ in test_loader:
        lr_images = lr_images.cuda()
        outputs = model(lr_images)
        test_predictions.append(outputs.cpu())

test_predictions = torch.cat(test_predictions)


In [ ]:
output_folder = '/kaggle/working/test_predictions2'
os.makedirs(output_folder, exist_ok=True)

for i, img in enumerate(test_predictions):
    img = img.permute(1, 2, 0).numpy()
    img = (img * 0.5) + 0.5  # Unnormalize
    img = (img * 255).astype(np.uint8)
    if i < 9:
        Image.fromarray(img).save(os.path.join(output_folder, f'test_0000{i+1}.png'))
    else :
        Image.fromarray(img).save(os.path.join(output_folder, f'test_000{i+1}.png'))


In [ ]:
# model.eval()
# output_folder = '/kaggle/working/test_predictions'
# os.makedirs(output_folder, exist_ok=True)

# with torch.no_grad():
#     for lr_images, _, filenames in test_loader:
#         lr_images = lr_images.cuda()
#         outputs = model(lr_images)
#         outputs = outputs.cpu()

#         for i, img in enumerate(outputs):
#             img = img.permute(1, 2, 0).numpy()
#             img = (img * 0.5) + 0.5  # Unnormalize
#             img = (img * 255).astype(np.uint8)
#             Image.fromarray(img).save(os.path.join(output_folder, filenames[i]))

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

folder_path = '/kaggle/working/test_predictions2'
output_csv = 'submission.csv'
images_to_csv(folder_path, output_csv)